In [1]:
import sys
sys.path.append('/home/ivan/distribution_connector')
import os

In [2]:
from sklearn import datasets
import numpy as np
import torch
from tqdm import tqdm
from connector_utils import test_models, gather_statistics, test_func
import matplotlib.pyplot as plt

In [3]:
from connector import Connector
# from one_layer_utils import samples, make_dataset, get_model, get_b
from utils import test_model
from flows.models.utils import test_flow

# MNIST

In [7]:
import models
architecture = getattr(models, "LinearOneLayer") #LinearOneLayer LogRegression

import data
loaders, num_classes = data.loaders(
    "MNIST",
    "data",
    1024,
    1,
    "VGG",
    True)

model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load('curves_mnist/LinearOneLayer/LongTraining/curve16/checkpoint-30.pt')['model_state'])
model2.load_state_dict(torch.load('curves_mnist/LinearOneLayer/LongTraining/curve17/checkpoint-30.pt')['model_state'])

import pickle
data_path = '/home/ivan/dnn-mode-connectivity/data/MNIST.pickle'
with open(data_path, 'rb') as handle:
    dataset, B = pickle.load(handle)
b2 = torch.FloatTensor([np.array(B).mean(0)])
b = np.array(B).mean(0)

train_dataset, test_dataset = dataset[:-8000], dataset[-8000:]
modes = [dataset[-4000:-2000], dataset[-2000:]]

You are going to run models on the test set. Are you sure?


In [12]:
te_list = []
tr_list = []
for i in range(1, 20):
    path = 'curves_mnist/LinearOneLayer/LongTraining/curve'+str(i)+'/checkpoint-30.pt'
    model1.load_state_dict(torch.load(path)['model_state'])
    model1.cuda()
    tr, te = test_model(model1, loaders, cuda=True)
    te_list.append(te['accuracy'])
    tr_list.append(tr['accuracy'])
    
te_list = np.array(te_list)
tr_list = np.array(tr_list)

train results {'nll': 0.03891415004928907, 'loss': 0.03891415004928907, 'accuracy': 99.14333333333333}
test results {'nll': 0.06730707376003266, 'loss': 0.06730707376003266, 'accuracy': 97.95}
train results {'nll': 0.039054598027467725, 'loss': 0.039054598027467725, 'accuracy': 99.145}
test results {'nll': 0.06708879473209381, 'loss': 0.06708879473209381, 'accuracy': 98.03}
train results {'nll': 0.03940047723849614, 'loss': 0.03940047723849614, 'accuracy': 99.12833333333333}
test results {'nll': 0.06816846108436585, 'loss': 0.06816846108436585, 'accuracy': 97.98}
train results {'nll': 0.039004734847942986, 'loss': 0.039004734847942986, 'accuracy': 99.13666666666667}
test results {'nll': 0.06703689527511597, 'loss': 0.06703689527511597, 'accuracy': 97.99}
train results {'nll': 0.0389949737071991, 'loss': 0.0389949737071991, 'accuracy': 99.15166666666667}
test results {'nll': 0.0677187137722969, 'loss': 0.0677187137722969, 'accuracy': 97.99}
train results {'nll': 0.038988523644208906, 'l

In [13]:
te_list = np.array(te_list)
tr_list = np.array(tr_list)

In [14]:
te_list.mean(), te_list.std(), tr_list.mean(), tr_list.std()

(98.01578947368422,
 0.030142781185845187,
 99.14026315789475,
 0.010047593728614407)

In [5]:
cntr = Connector(None, None)

# MNIST PCA

In [6]:
# import models
# architecture = getattr(models, "LinearOneLayer") #LinearOneLayer LogRegression

# import data
# loaders, num_classes = data.loaders(
#     "MNIST",
#     "data",
#     1024,
#     1,
#     "VGG",
#     True)

# import pickle
# data_path = '/home/ivan/dnn-mode-connectivity/data/MNIST_PCA.pickle'
# with open(data_path, 'rb') as handle:
#     dataset, B, transformation = pickle.load(handle)
# b2 = torch.FloatTensor([np.array(B).mean(0)])
# b = np.array(B).mean(0)

# train_dataset, test_dataset = dataset[:-8000], dataset[-8000:]
# modes = [dataset[-4000:-2000], dataset[-2000:]]

In [7]:
test_dataset.shape

(8000, 795)

In [8]:
# from flows.models import RealNVP, RealNVP_OneLayer_full, IAF_OneLayer_full
# flow = RealNVP_OneLayer_full(in_dim=80, dim_middle=200, N_layers=5, batch_norm=False, data_b2=b, transPSA=transformation)
# flow.load_state_dict(torch.load('/home/ivan/distribution_connector/flow_models/RealNVP/MNIST_PCA_bijection3/checkpoint-2.pt')['model_state'])


In [9]:
# flow.cuda();
# test_flow(flow, architecture, loaders, b2, cntr=None, verbose=True,
#               test_sampling=True, test_flow=False, cuda=True, transPSA=transformation)

In [10]:
# flow.cuda();
# stat_dict, stat_dict_mean, stat_dict_std = gather_statistics(dataset[-8000:], 
#                                                              loaders, b2, architecture, 
#                                                              flow=flow, K=80, t=0.5, 
#                                                              verbose=True, func_name=['flow_connect'],
#                                                              save_stat=True, path='stat/MNIST/bijectionPCA',
#                                                              transPSA=transformation)

# Gavering sample and error time plots on MNIST

In [11]:
funcs = [func for func in dir(cntr) if callable(getattr(cntr, func)) and 'connect' in func 
         and 'flow' not in func and 'third_cumulant' not in func]

In [12]:
funcs

['arc_connect',
 'arc_connect_PCA',
 'inverse_connect',
 'inverse_connect_PCA',
 'lin_connect',
 'simul_diag_connect']

In [13]:
# for func_name in funcs:
#     t = np.linspace(0,1, 61)[1:-1]
#     stat = test_func(modes, loaders, b, architecture, func_name, flow=None, K=80, t=t, 
#                      show=False, verbose=False, save_pic=True, path='MNIST', save_samples=False,
#                      test_models=True, compute_distribution_stat=False, show_type='hex')
#     t = np.linspace(0,1, 5)[1:-1]
#     stat = test_func(modes, loaders, b, architecture, func_name, flow=None, K=80, t=t, 
#                      show=False, verbose=False, save_pic=False, path='MNIST', save_samples=True)
    

## flow 

In [14]:
# '/home/ivan/distribution_connector/flow_models/Flow_IAF/MNIST_flow/checkpoint-150.pt'
# from flows.models import RealNVP, RealNVP_OneLayer_full, IAF_OneLayer_full
# flow = IAF_OneLayer_full(in_dim=795, dim_middle=795, N_layers=2, batch_norm=False, data_b2=b)
# flow.load_state_dict(torch.load('/home/ivan/distribution_connector/flow_models/Flow_IAF/MNIST_flow/checkpoint-120.pt')['model_state'])


# flow.cuda();


In [17]:
from flows.models import RealNVP, RealNVP_OneLayer_full

flow = RealNVP_OneLayer_full(in_dim=795, dim_middle=2000, N_layers=1, batch_norm=True, data_b2=b)
flow.load_state_dict(torch.load('/home/ivan/distribution_connector/flow_models/RealNVP/MNIST_bijection/checkpoint-20.pt')['model_state'])
flow.cuda();


mask len 2


In [ ]:
t = np.linspace(0,1, 61)[1:-1]
stat = test_func(modes, loaders, b2, architecture, 'flow_connect', flow=flow, K=80, t=t, 
                     show=False, verbose=False, save_pic=True, path='MNIST', save_samples=False,
                     test_models=True, compute_distribution_stat=False, show_type='hex')

# Computing mean and std for different methods on MNIST

## flow 

In [36]:
# checkpoint-20.pt -- top for MNIST bijection
# checkpoint-7 top for CIFAR bijection
# checkpoint-120 top for MNIST Flow IAF 

In [61]:
# '/home/ivan/distribution_connector/flow_models/Flow_IAF/MNIST_flow/checkpoint-150.pt'
# from flows.models import RealNVP, RealNVP_OneLayer_full, IAF_OneLayer_full
# flow = IAF_OneLayer_full(in_dim=795, dim_middle=795, N_layers=2, batch_norm=False, data_b2=b)
# flow.load_state_dict(torch.load('/home/ivan/distribution_connector/flow_models/Flow_IAF/MNIST_flow/checkpoint-120.pt')['model_state'])





In [57]:
# from flows.models import RealNVP, RealNVP_OneLayer_full

# flow = RealNVP_OneLayer_full(in_dim=795, dim_middle=795, N_layers=1, batch_norm=True, data_b2=b)
# flow.load_state_dict(torch.load('/home/ivan/distribution_connector/flow_models/RealNVP/MNIST_flow1/checkpoint-8.pt')['model_state'])



In [58]:
# model = flow

In [59]:
# b = b2
# cuda = True
# verbose = True
# transPSA = None
# model.eval()
# res = model.sample(K=2000).cpu().data.numpy()
# if transPSA is not None:
#     res = transPSA.inverse_transform(res)
# m = get_model(res, b, architecture)
# if cuda:
#     m.cuda()
# result_s = test_model(m, loaders, verbose=verbose, cuda=cuda)

In [60]:
flow.cuda();
test_flow(flow, architecture, loaders, b2, cntr=None, verbose=True,
              test_sampling=True, test_flow=False, cuda=True, transPSA=None)

sampling testing
train results {'nll': 0.1873208567937215, 'loss': 0.1873208567937215, 'accuracy': 94.21833333333333}
test results {'nll': 0.19890189328193664, 'loss': 0.19890189328193664, 'accuracy': 94.02}


(({'nll': 0.1873208567937215,
   'loss': 0.1873208567937215,
   'accuracy': 94.21833333333333},
  {'nll': 0.19890189328193664,
   'loss': 0.19890189328193664,
   'accuracy': 94.02}),
 None)

In [62]:
# flow.cuda();
# stat_dict, stat_dict_mean, stat_dict_std = gather_statistics(dataset[-8000:], 
#                                                              loaders, b2, architecture, 
#                                                              flow=flow, K=80, t=0.5, 
#                                                              verbose=True, func_name=['flow_connect'],
#                                                              save_stat=True, path='stat/MNIST/IAFflow')

## basic 

In [17]:
test_dataset = dataset[-8000:]

In [18]:
funcs = [func for func in dir(cntr) if callable(getattr(cntr, func)) and 'connect' in func 
         and 'flow' not in func and 'third_cumulant' in func]

In [19]:
stat_dict, stat_dict_mean, stat_dict_std = gather_statistics(test_dataset, 
                                                             loaders, b2, architecture, 
                                                             flow=None, K=80, t=0.5, 
                                                             verbose=True, func_name=funcs,
                                                             save_stat=True, path='stat/MNIST/third_cumulant')


  0%|          | 0/4 [00:00<?, ?it/s]

N_models 4
third_cumulant_connect
c 1600.3411257129785 512000
0.5
train results {'nll': 0.8071608489990234, 'loss': 0.8071608489990234, 'accuracy': 75.44}
test results {'nll': 0.788148390865326, 'loss': 0.788148390865326, 'accuracy': 76.12}
third_cumulant_connect
c 1594.631502638252 512000
0.5
train results {'nll': 0.6341616157531739, 'loss': 0.6341616157531739, 'accuracy': 78.96333333333334}
test results {'nll': 0.6215544948577881, 'loss': 0.6215544948577881, 'accuracy': 79.51}
third_cumulant_connect
c 1602.5800297365204 512000
0.5



 25%|██▌       | 1/4 [01:37<04:51, 97.16s/it]

train results {'nll': 0.7839152718226114, 'loss': 0.7839152718226114, 'accuracy': 72.36}
test results {'nll': 0.7712775550842285, 'loss': 0.7712775550842285, 'accuracy': 72.52}
third_cumulant_connect
c 1601.0657202684679 512000
0.5
train results {'nll': 0.7509037694613139, 'loss': 0.7509037694613139, 'accuracy': 74.60666666666667}
test results {'nll': 0.7424520283699035, 'loss': 0.7424520283699035, 'accuracy': 74.86}
third_cumulant_connect
c 1617.6414617463297 512000
0.5



 50%|█████     | 2/4 [02:45<02:57, 88.53s/it]

train results {'nll': 0.8341048234621684, 'loss': 0.8341048234621684, 'accuracy': 73.07}
test results {'nll': 0.8145449273109436, 'loss': 0.8145449273109436, 'accuracy': 73.45}
third_cumulant_connect
c 1609.567649403868 512000
0.5



100%|██████████| 4/4 [03:20<00:00, 50.19s/it]

train results {'nll': 0.8697093101183574, 'loss': 0.8697093101183574, 'accuracy': 69.51166666666667}
test results {'nll': 0.8651455919265747, 'loss': 0.8651455919265747, 'accuracy': 69.71}
saving to  data/stat/MNIST/third_cumulanterror_stat.pickle


In [24]:
stat_dict_mean, stat_dict_std

({'train': {'arc_connect': 97.89348148148149,
   'arc_connect_PCA': 97.66703703703703,
   'inverse_connect': 97.84737037037036,
   'inverse_connect_PCA': 97.46407407407409,
   'lin_connect': 96.54114814814814,
   'simul_diag_connect': 97.84322222222224},
  'test': {'arc_connect': 97.03577777777775,
   'arc_connect_PCA': 97.02199999999999,
   'inverse_connect': 96.96955555555556,
   'inverse_connect_PCA': 96.81,
   'lin_connect': 95.87822222222223,
   'simul_diag_connect': 97.00844444444445}},
 {'train': {'arc_connect': 0.11143166107154394,
   'arc_connect_PCA': 0.09728974727356571,
   'inverse_connect': 0.13658556260530705,
   'inverse_connect_PCA': 0.11721224578692356,
   'lin_connect': 0.41123118700219913,
   'simul_diag_connect': 0.11861827977296019},
  'test': {'arc_connect': 0.14210307509205192,
   'arc_connect_PCA': 0.13511476603243608,
   'inverse_connect': 0.16937736377103318,
   'inverse_connect_PCA': 0.16152055252780917,
   'lin_connect': 0.40518467059348473,
   'simul_diag_c

# CIFAR10

In [4]:
import models
architecture = getattr(models, "LinearOneLayerCF") #LinearOneLayer LogRegression

import data
loaders, num_classes = data.loaders(
    "CIFAR10",
    "data",
    1024,
    1,
    "VGG",
    False)

model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load('curves/LinearOneLayer/curve16/checkpoint-400.pt')['model_state'])
model2.load_state_dict(torch.load('curves/LinearOneLayer/curve17/checkpoint-400.pt')['model_state'])

import pickle
data_path = '/home/ivan/dnn-mode-connectivity/data/CIFAR.pickle'
with open(data_path, 'rb') as handle:
    dataset, B = pickle.load(handle)
b2 =  torch.FloatTensor([np.array(B).mean(0)])
b = np.array(B).mean(0)

train_dataset, test_dataset = dataset[:-8000], dataset[-8000:]
modes = [dataset[-4000:-2000], dataset[-2000:]]

Files already downloaded and verified
You are going to run models on the test set. Are you sure?
Files already downloaded and verified


In [13]:
from sklearn.decomposition import PCA
transformation = PCA(n_components=80, svd_solver='full', random_state=1)
transformation.fit(dataset)

PCA(copy=True, iterated_power='auto', n_components=80, random_state=1,
  svd_solver='full', tol=0.0, whiten=False)

In [14]:
dataset_PCA = transformation.transform(dataset)

In [15]:
dataset_PCA.shape, dataset.shape

((40000, 80), (40000, 3083))

In [20]:
def PCA_TR(model):
    print(model.shape, transformation.mean_.shape)
    model = model - transformation.mean_
    print(model.shape, transformation.components_.shape)
    model = model @ transformation.components_.T
    return model

In [21]:
dataset_PCA2 = PCA_TR(dataset)

(40000, 3083) (3083,)
(40000, 3083) (80, 3083)


In [25]:
dataset_PCA

array([[ 0.08025937,  0.26700705,  0.03110478, ..., -0.02661011,
        -0.15267825, -0.01615445],
       [ 0.01122742,  0.02091683,  0.03171532, ...,  0.03098021,
         0.00724024,  0.02485646],
       [-0.24867547,  0.42444724, -0.03872361, ...,  0.06727547,
         0.10987452, -0.0476301 ],
       ...,
       [-0.07434984,  0.05637497,  0.06242565, ..., -0.01822452,
        -0.01985219,  0.03664694],
       [ 0.1393429 ,  0.28890634, -0.07347254, ...,  0.06800801,
         0.01303523, -0.02296821],
       [-0.42998293,  0.48278493,  0.18912846, ..., -0.05500802,
         0.06839504,  0.06823014]], dtype=float32)

In [24]:
dataset_PCA2

array([[ 0.08025937,  0.26700705,  0.03110478, ..., -0.02661011,
        -0.15267825, -0.01615445],
       [ 0.01122742,  0.02091683,  0.03171532, ...,  0.03098021,
         0.00724024,  0.02485646],
       [-0.24867547,  0.42444724, -0.03872361, ...,  0.06727547,
         0.10987452, -0.0476301 ],
       ...,
       [-0.07434984,  0.05637497,  0.06242565, ..., -0.01822452,
        -0.01985219,  0.03664694],
       [ 0.1393429 ,  0.28890634, -0.07347254, ...,  0.06800801,
         0.01303523, -0.02296821],
       [-0.42998293,  0.48278493,  0.18912846, ..., -0.05500802,
         0.06839504,  0.06823014]], dtype=float32)

In [17]:
te_list = []
tr_list = []
for i in range(1, 20):
    path = 'curves/LinearOneLayer/curve'+str(i)+'/checkpoint-400.pt'
    model1.load_state_dict(torch.load(path)['model_state'])
    model1.cuda()
    tr, te = test_model(model1, loaders, cuda=True)
    te_list.append(te['accuracy'])
    tr_list.append(tr['accuracy'])
    
te_list = np.array(te_list)
tr_list = np.array(tr_list)

train results {'nll': 0.8496363522338867, 'loss': 0.8496363522338867, 'accuracy': 70.452}
test results {'nll': 1.3088677494049072, 'loss': 1.3088677494049072, 'accuracy': 58.83}
train results {'nll': 0.8452534885978699, 'loss': 0.8452534885978699, 'accuracy': 70.65}
test results {'nll': 1.298913423347473, 'loss': 1.298913423347473, 'accuracy': 59.53}
train results {'nll': 0.8480340089988708, 'loss': 0.8480340089988708, 'accuracy': 70.538}
test results {'nll': 1.3227679414749145, 'loss': 1.3227679414749145, 'accuracy': 59.11}
train results {'nll': 0.8484666329956054, 'loss': 0.8484666329956054, 'accuracy': 70.484}
test results {'nll': 1.3180574544906616, 'loss': 1.3180574544906616, 'accuracy': 59.4}
train results {'nll': 0.8432427541923523, 'loss': 0.8432427541923523, 'accuracy': 70.814}
test results {'nll': 1.3117229124069214, 'loss': 1.3117229124069214, 'accuracy': 59.09}
train results {'nll': 0.8470813488197326, 'loss': 0.8470813488197326, 'accuracy': 70.676}
test results {'nll': 1.3

In [18]:
te_list.mean(), te_list.std(), tr_list.mean(), tr_list.std()

(59.123684210526314,
 0.26230176527519233,
 70.60947368421053,
 0.12687970615427344)

# Gavering sample and error time plots on CIFAR10

In [5]:
# for func_name in funcs:
#     t = np.linspace(0,1, 61)[1:-1]
#     stat = test_func(modes, loaders, b2, architecture, func_name, flow=None, K=400, t=t, 
#                      show=False, verbose=False, save_pic=True, path='CIFAR', save_samples=False)
#     t = np.linspace(0,1, 5)[1:-1]
#     stat = test_func(modes, loaders, b2, architecture, func_name, flow=None, K=400, t=t, 
#                      show=False, verbose=False, save_pic=False, path='CIFAR', save_samples=True)

## RNVP flow

In [6]:
# t = np.linspace(0,1, 61)[1:-1]
# stat = test_func(modes, loaders, b2, architecture, func_name='flow_connect', flow=flow, K=400, t=t, 
#                  show=False, verbose=False, save_pic=True, test_models=True, path='CIFAR', save_samples=False)

In [83]:
# t = np.linspace(0,1, 5)[1:-1]
# stat = test_func(modes, loaders, b2, architecture, func_name='flow_connect', flow=flow, K=400, t=t, 
#                  show=False, verbose=False, save_pic=False, path='CIFAR', save_samples=True)

# Computing mean and std for different methods on CIFAR10

In [14]:
model1.cuda()
test_model(model1, loaders, cuda=True)

train results {'nll': 0.8489676392555237, 'loss': 0.8489676392555237, 'accuracy': 70.572}
test results {'nll': 1.313883560180664, 'loss': 1.313883560180664, 'accuracy': 58.9}


({'nll': 0.8489676392555237, 'loss': 0.8489676392555237, 'accuracy': 70.572},
 {'nll': 1.313883560180664, 'loss': 1.313883560180664, 'accuracy': 58.9})

## flow

In [87]:
from flows.models import RealNVP, RealNVP_OneLayer_full, IAF_OneLayer_full

flow = RealNVP_OneLayer_full(in_dim=3083, dim_middle=3000, N_layers=1, batch_norm=False, data_b2=b)
flow.load_state_dict(torch.load('/home/ivan/distribution_connector/flow_models/RealNVP/CIFAR_test1/checkpoint-7.pt')['model_state'])


mask len 2


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [88]:
# from flows.models import RealNVP, RealNVP_OneLayer_full, IAF_OneLayer_full
# flow = IAF_OneLayer_full(in_dim=3083, dim_middle=3000, N_layers=1, batch_norm=False, data_b2=b)
# flow.load_state_dict(torch.load('/home/ivan/distribution_connector/flow_models/RealNVP/CIFAR_test1/checkpoint-7.pt')['model_state'])

In [89]:
flow.cuda();
stat_dict, stat_dict_mean, stat_dict_std = gather_statistics(dataset[-8000:], 
                                                             loaders, b2, architecture, 
                                                             flow=flow, K=80, t=0.5, 
                                                             verbose=True, func_name=['flow_connect'],
                                                             save_stat=True, path='stat/CIFAR/bijection')



  0%|          | 0/4 [00:00<?, ?it/s]

N_models 4
flow_connect
0.5
train results {'nll': 1.0533692544174194, 'loss': 1.0533692544174194, 'accuracy': 63.456}
test results {'nll': 1.6187260831832886, 'loss': 1.6187260831832886, 'accuracy': 53.59}
flow_connect
0.5
train results {'nll': 1.0545070586013794, 'loss': 1.0545070586013794, 'accuracy': 63.1}
test results {'nll': 1.7111023202896118, 'loss': 1.7111023202896118, 'accuracy': 52.64}
flow_connect
0.5




 25%|██▌       | 1/4 [01:00<03:01, 60.47s/it]

train results {'nll': 1.0488071635627747, 'loss': 1.0488071635627747, 'accuracy': 63.788}
test results {'nll': 1.5592954841613769, 'loss': 1.5592954841613769, 'accuracy': 55.26}
flow_connect
0.5
train results {'nll': 1.0633495126342773, 'loss': 1.0633495126342773, 'accuracy': 63.462}
test results {'nll': 1.5593252670288087, 'loss': 1.5593252670288087, 'accuracy': 54.64}
flow_connect
0.5




 50%|█████     | 2/4 [01:41<01:49, 54.79s/it]

train results {'nll': 1.0475879463577271, 'loss': 1.0475879463577271, 'accuracy': 63.818}
test results {'nll': 1.5472380882263184, 'loss': 1.5472380882263184, 'accuracy': 54.58}
flow_connect
0.5




 75%|███████▌  | 3/4 [02:03<00:44, 44.90s/it]

100%|██████████| 4/4 [02:03<00:00, 30.96s/it]

train results {'nll': 1.0577251321029664, 'loss': 1.0577251321029664, 'accuracy': 63.144}
test results {'nll': 1.6692164546966552, 'loss': 1.6692164546966552, 'accuracy': 52.93}
saving to  data/stat/CIFAR/bijectionerror_stat.pickle


## basic 

In [7]:
test_dataset = dataset[-8000:]

In [11]:
funcs = [func for func in dir(cntr) if callable(getattr(cntr, func)) and 'connect' in func 
         and 'flow' not in func and 'third_cumulant' in func]

In [12]:
funcs

['third_cumulant_connect']

In [14]:
stat_dict, stat_dict_mean, stat_dict_std = gather_statistics(test_dataset, 
                                                             loaders, b2, architecture, 
                                                             flow=None, K=200, t=0.5, 
                                                             verbose=True, func_name=funcs,
                                                             save_stat=True, path='stat/CIFAR/third_cumulant')


  0%|          | 0/4 [00:00<?, ?it/s]

N_models 4
third_cumulant_connect
c 49339.753945420765 8000000
0.5
train results {'nll': 192.03328536132813, 'loss': 192.03328536132813, 'accuracy': 10.95}
test results {'nll': 159.18884404296875, 'loss': 159.18884404296875, 'accuracy': 11.4}
third_cumulant_connect
c 49147.18525962689 8000000
0.5
train results {'nll': 197.92421240234376, 'loss': 197.92421240234376, 'accuracy': 10.81}
test results {'nll': 172.68769267578125, 'loss': 172.68769267578125, 'accuracy': 11.54}
third_cumulant_connect
c 49338.789716830164 8000000
0.5



 25%|██▌       | 1/4 [19:02<57:07, 1142.60s/it]

train results {'nll': 200.2789377734375, 'loss': 200.2789377734375, 'accuracy': 11.184}
test results {'nll': 174.5269504638672, 'loss': 174.5269504638672, 'accuracy': 12.38}
third_cumulant_connect
c 49031.59461684667 8000000
0.5
train results {'nll': 196.78189912109374, 'loss': 196.78189912109374, 'accuracy': 10.758}
test results {'nll': 166.34349104003905, 'loss': 166.34349104003905, 'accuracy': 11.96}
third_cumulant_connect
c 49216.40067193843 8000000
0.5



 50%|█████     | 2/4 [32:55<34:59, 1049.84s/it]

train results {'nll': 203.76456399902344, 'loss': 203.76456399902344, 'accuracy': 10.178}
test results {'nll': 165.93680776367188, 'loss': 165.93680776367188, 'accuracy': 11.31}
third_cumulant_connect
c 49634.76412962114 8000000
0.5



100%|██████████| 4/4 [39:04<00:00, 586.23s/it]

train results {'nll': 204.2034564892578, 'loss': 204.2034564892578, 'accuracy': 10.938}
test results {'nll': 168.1272219970703, 'loss': 168.1272219970703, 'accuracy': 12.78}
saving to  data/stat/CIFAR/third_cumulanterror_stat.pickle


# Checking

In [23]:
with open('data/stat/CIFAR/third_cumulanterror_stat.pickle', 'rb') as handle:
        load = pickle.load(handle)

In [24]:
load[1]

{'train': {'third_cumulant_connect': 10.802999999999999},
 'test': {'third_cumulant_connect': 11.895000000000001}}

In [25]:
load[2]

{'train': {'third_cumulant_connect': 0.31029394665918486},
 'test': {'third_cumulant_connect': 0.5378893938348291}}

In [26]:
stat_dict_mean, stat_dict_std

({'train': {'flow_connect': 63.60966666666667},
  'test': {'flow_connect': 53.94}},
 {'train': {'flow_connect': 0.353156087618808},
  'test': {'flow_connect': 0.9552137631615929}})